# EDA, Handling Categorical Values and Handling Missing Values

## Imports & Load Data:

In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#Complete Route to proyect directory
os.chdir("/media/roovedot/common/VSrootWorkspace/House-Price-Predictions-with-Random-Forest-Regression-Model")
#Load data from Train file
housing = pd.read_csv("data/train.csv")
housing.head() #Preview of data to check correct Load


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## Getting to know the Data:


### Basic Info

In [2]:

housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

### Distributions for every Column

In [ ]:
for col in housing.select_dtypes(include=['int64', 'float64']).columns:
    plt.figure(figsize=(10, 6))  # Set figure size

    sns.displot(housing[col])
    
    # Set plot title and labels
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    
    # Display the plot
    plt.show()

## Initial Correlations:

In [4]:
# Select all numeric columns
hous_num = housing.select_dtypes(include=['float64', 'int64'])

# Get each Feature's correlation with 'SalePrice'
hous_num_corr = hous_num.corr()['SalePrice'].sort_values(ascending=False)  # Ordenar de mayor a menor

# Print all correlations in order
for var, corr in hous_num_corr.items():
    print(f"{var}: {corr:.3f}") # corr:.2f formats corr values to 2 decimals

SalePrice: 1.000
OverallQual: 0.791
GrLivArea: 0.709
GarageCars: 0.640
GarageArea: 0.623
TotalBsmtSF: 0.614
1stFlrSF: 0.606
FullBath: 0.561
TotRmsAbvGrd: 0.534
YearBuilt: 0.523
YearRemodAdd: 0.507
GarageYrBlt: 0.486
MasVnrArea: 0.477
Fireplaces: 0.467
BsmtFinSF1: 0.386
LotFrontage: 0.352
WoodDeckSF: 0.324
2ndFlrSF: 0.319
OpenPorchSF: 0.316
HalfBath: 0.284
LotArea: 0.264
BsmtFullBath: 0.227
BsmtUnfSF: 0.214
BedroomAbvGr: 0.168
ScreenPorch: 0.111
PoolArea: 0.092
MoSold: 0.046
3SsnPorch: 0.045
BsmtFinSF2: -0.011
BsmtHalfBath: -0.017
MiscVal: -0.021
Id: -0.022
LowQualFinSF: -0.026
YrSold: -0.029
OverallCond: -0.078
MSSubClass: -0.084
EnclosedPorch: -0.129
KitchenAbvGr: -0.136


## Categorical Values:

### Encoding Categorical Values (with One-hot Encoding)

In [5]:
house_cat = housing.select_dtypes(include=['object'])  # Select categorical columns
house_cat_encoded = pd.get_dummies(house_cat, dummy_na=True)  # One-hot encode categorical variables. dummy_na creates a column indicating missing value

# Join "Saleprice" Target to hous_cat_encoded
# axis=1 Ensures SalePrice gets Passed as a Column and not a row
house_cat_encoded = pd.concat([house_cat_encoded, housing["SalePrice"]], axis=1) 

# Get correlations of each Category with the Target
house_cat_encoded_corr = house_cat_encoded.corr()['SalePrice'].sort_values(ascending=False)  # Ordenar de mayor a menor

# Print all correlations in order
for var, corr in house_cat_encoded_corr.items():
    print(f"{var}: {corr:.3f}") # corr:.2f formats corr values to 2 decimals'''


SalePrice: 1.000
BsmtQual_Ex: 0.553
KitchenQual_Ex: 0.504
Foundation_PConc: 0.498
ExterQual_Gd: 0.452
ExterQual_Ex: 0.451
BsmtFinType1_GLQ: 0.435
HeatingQC_Ex: 0.435
GarageFinish_Fin: 0.420
Neighborhood_NridgHt: 0.402
SaleType_New: 0.358
SaleCondition_Partial: 0.352
FireplaceQu_Gd: 0.339
GarageType_Attchd: 0.336
MasVnrType_Stone: 0.330
Neighborhood_NoRidge: 0.330
KitchenQual_Gd: 0.322
BsmtExposure_Gd: 0.307
Exterior2nd_VinylSd: 0.306
Exterior1st_VinylSd: 0.305
GarageCond_TA: 0.276
FireplaceQu_Ex: 0.255
CentralAir_Y: 0.251
GarageQual_TA: 0.245
MSZoning_RL: 0.245
HouseStyle_2Story: 0.243
Electrical_SBrkr: 0.242
RoofStyle_Hip: 0.236
GarageType_BuiltIn: 0.235
BsmtQual_Gd: 0.235
PavedDrive_Y: 0.232
LotShape_IR1: 0.223
Neighborhood_StoneBr: 0.215
MasVnrType_BrkFace: 0.198
Fence_nan: 0.172
GarageFinish_RFn: 0.170
RoofMatl_WdShngl: 0.169
FireplaceQu_TA: 0.163
PoolQC_Ex: 0.144
LotConfig_CulDSac: 0.142
Neighborhood_Somerst: 0.140
BldgType_1Fam: 0.137
BsmtExposure_Av: 0.137
Exterior1st_CemntBd: 0

In [6]:
house_cat_encoded.columns#.tolist()

Index(['MSZoning_C (all)', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL',
       'MSZoning_RM', 'MSZoning_nan', 'Street_Grvl', 'Street_Pave',
       'Street_nan', 'Alley_Grvl',
       ...
       'SaleType_WD', 'SaleType_nan', 'SaleCondition_Abnorml',
       'SaleCondition_AdjLand', 'SaleCondition_Alloca', 'SaleCondition_Family',
       'SaleCondition_Normal', 'SaleCondition_Partial', 'SaleCondition_nan',
       'SalePrice'],
      dtype='object', length=295)

### Keeping only Valuable Categories:

In [7]:
# Drop all Columns which have an Correlation below 0.4 Absolute Value
for col, corr in house_cat_encoded_corr.items():
    # If the absolute value of the correlation is less than 0.4
    
    if (abs(corr) < 0.4 and col in house_cat_encoded.columns) or pd.isna(corr):
        # inplace=True ensures operating on the original Dataframe
        # axis=1 tells drop() method we are dropping columns and not rows
        house_cat_encoded.drop(columns=col, axis=1, inplace=True)  # Drop Column

# Remove 'SalePrice' before merging with the original DataFrame
house_cat_encoded.drop(columns='SalePrice', axis=1, inplace=True)

# Convert all boolean columns in the DataFrame to integer type for compatibility
house_cat_encoded = house_cat_encoded.astype(int)

house_cat_encoded.columns

Index(['Neighborhood_NridgHt', 'ExterQual_Ex', 'ExterQual_Gd', 'ExterQual_TA',
       'Foundation_PConc', 'BsmtQual_Ex', 'BsmtQual_TA', 'BsmtFinType1_GLQ',
       'HeatingQC_Ex', 'KitchenQual_Ex', 'KitchenQual_TA', 'FireplaceQu_nan',
       'GarageFinish_Fin', 'GarageFinish_Unf'],
      dtype='object')

### Merge Encoded Data:

In [8]:
# Concatenate Selected encoded categorical variables with the original dataframe
housing = pd.concat([housing, house_cat_encoded], axis=1)

# Drop the original categorical columns to avoid redundancy
housing = housing.drop(columns=house_cat.columns)

housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 52 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Id                    1460 non-null   int64  
 1   MSSubClass            1460 non-null   int64  
 2   LotFrontage           1201 non-null   float64
 3   LotArea               1460 non-null   int64  
 4   OverallQual           1460 non-null   int64  
 5   OverallCond           1460 non-null   int64  
 6   YearBuilt             1460 non-null   int64  
 7   YearRemodAdd          1460 non-null   int64  
 8   MasVnrArea            1452 non-null   float64
 9   BsmtFinSF1            1460 non-null   int64  
 10  BsmtFinSF2            1460 non-null   int64  
 11  BsmtUnfSF             1460 non-null   int64  
 12  TotalBsmtSF           1460 non-null   int64  
 13  1stFlrSF              1460 non-null   int64  
 14  2ndFlrSF              1460 non-null   int64  
 15  LowQualFinSF         

## Missing Values:

Most of the missing Values were on categorical Columns, which we have already handled with One-Hot Encoding, even getting valuable info from missing values, like the case of FireplaceQu_nan, with a correlation of -0.472

In [9]:
# 5 Columns with the most missing Values
housing.isnull().sum().sort_values(ascending=False).head()

LotFrontage     259
GarageYrBlt      81
MasVnrArea        8
Id                0
ExterQual_Ex      0
dtype: int64

The missing values in LotFrontage, GarageYrBlt and MasVnrArea probably mean these features are not on the house.  

Since I think them missing is actually valuable information, I will create a boolean column for each one indicating if the value is missing, and I will set the value of the feature to 0 except for GarageYrBlt.

### Handling Missing Values:

In [10]:
# WARNING: If you execute this cell more than once, it will set all the indicator columns to 0, making it useless

housing_na = housing[["LotFrontage", "GarageYrBlt", "MasVnrArea"]]

for column in housing_na.columns:
    if column == "GarageYrBlt": # For GarageYrBlt, we will set the value to the mean instead of 0
        #Create column indicating if value was missing
        housing[f'{column}_nan'] = housing[column].isnull().astype(int)

        # Impute missing values with value 0
        housing[column].fillna(housing["GarageYrBlt"].mean(), inplace=True)
    else:
        #Create column indicating if value was missing
        housing[f'{column}_nan'] = housing[column].isnull().astype(int)

        # Impute missing values with value 0
        housing[column].fillna(0, inplace=True)

housing.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 55 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Id                    1460 non-null   int64  
 1   MSSubClass            1460 non-null   int64  
 2   LotFrontage           1460 non-null   float64
 3   LotArea               1460 non-null   int64  
 4   OverallQual           1460 non-null   int64  
 5   OverallCond           1460 non-null   int64  
 6   YearBuilt             1460 non-null   int64  
 7   YearRemodAdd          1460 non-null   int64  
 8   MasVnrArea            1460 non-null   float64
 9   BsmtFinSF1            1460 non-null   int64  
 10  BsmtFinSF2            1460 non-null   int64  
 11  BsmtUnfSF             1460 non-null   int64  
 12  TotalBsmtSF           1460 non-null   int64  
 13  1stFlrSF              1460 non-null   int64  
 14  2ndFlrSF              1460 non-null   int64  
 15  LowQualFinSF         

## Updated Correlations:

In [ ]:
housing.info() #Check data is properly transformed

In [ ]:
# Get each Feature's correlation with 'SalePrice'
housing_corr = housing.corr()['SalePrice'].sort_values(ascending=False)  # Ordenar de mayor a menor

# Print all correlations in order
for var, corr in housing_corr.items():
    print(f"{var}: {corr:.3f}") # corr:.2f formats corr values to 2 decimals

## Saving File for the Next Step of Cleaning: 

In [ ]:
housing.info()

In [8]:
housing.to_csv('data/train_catH_naH.csv')